<a href="https://colab.research.google.com/github/walkerjian/DailyCode/blob/main/ConGame.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

To implement Conway's Game of Life using the MVC (Model-View-Controller) paradigm, we'll structure our code as follows:

1. **Model**: Represents the state and logic of the Game of Life. It will have methods to update the state of the board based on the rules and to check the current state of a cell.
2. **View**: Responsible for displaying the current state of the board. It will print out the board from the top-leftmost live cell to the bottom-rightmost live cell.
3. **Controller**: Acts as an intermediary between the Model and the View. It initializes the board, starts the game, and updates the board for a given number of steps.

Let's start by implementing the Model.

### Model

The `GameOfLifeModel` class will:

1. Maintain the current state of the board.
2. Provide methods to get the state of a particular cell.
3. Have a method to update the board state based on the rules of the game.

Let's implement this class:

```python
class GameOfLifeModel:
    """Represents the model in Conway's Game of Life.
    
    The model is responsible for storing the current state of the game and
    providing logic to update that state based on the rules of the game.
    """
    
    def __init__(self, live_cells):
        """Initialize the model with a set of live cell coordinates."""
        self.live_cells = set(live_cells)
    
    def is_cell_alive(self, x, y):
        """Return True if the cell at (x, y) is alive, otherwise return False."""
        return (x, y) in self.live_cells
    
    def live_neighbours(self, x, y):
        """Return the number of live neighbours for the cell at (x, y)."""
        offsets = [(-1, -1), (-1, 0), (-1, 1), (0, -1), (0, 1), (1, -1), (1, 0), (1, 1)]
        count = 0
        for dx, dy in offsets:
            if self.is_cell_alive(x + dx, y + dy):
                count += 1
        return count
    
    def next_state(self):
        """Compute the next state of the board based on the current state."""
        new_live_cells = set()
        
        # Define the boundary for the region that needs to be checked
        if not self.live_cells:  # If no live cells, return empty set
            return new_live_cells
        
        min_x = min(cell[0] for cell in self.live_cells) - 1
        max_x = max(cell[0] for cell in self.live_cells) + 1
        min_y = min(cell[1] for cell in self.live_cells) - 1
        max_y = max(cell[1] for cell in self.live_cells) + 1
        
        for x in range(min_x, max_x + 1):
            for y in range(min_y, max_y + 1):
                live_neigh_count = self.live_neighbours(x, y)
                if self.is_cell_alive(x, y):
                    if live_neigh_count in [2, 3]:  # Live cell with 2 or 3 live neighbours remains alive
                        new_live_cells.add((x, y))
                else:
                    if live_neigh_count == 3:  # Dead cell with 3 live neighbours becomes alive
                        new_live_cells.add((x, y))
        
        self.live_cells = new_live_cells
```

With the model defined, let's move on to the View.

### View

The `GameOfLifeView` class will:

1. Display the current state of the board.
2. Print out the board from the top-leftmost live cell to the bottom-rightmost live cell.

Here's the implementation of the View:

```python
class GameOfLifeView:
    """Represents the view in Conway's Game of Life.
    
    The view is responsible for displaying the current state of the game board.
    """
    
    @staticmethod
    def display(live_cells):
        """Display the current state of the board using the provided live cell coordinates."""
        if not live_cells:
            print(".")
            return
        
        min_x = min(cell[0] for cell in live_cells)
        max_x = max(cell[0] for cell in live_cells)
        min_y = min(cell[1] for cell in live_cells)
        max_y = max(cell[1] for cell in live_cells)
        
        for y in range(min_y, max_y + 1):
            for x in range(min_x, max_x + 1):
                print('*' if (x, y) in live_cells else '.', end='')
            print()  # Move to the next line after each row
        print()  # Add an extra line for separation between steps
```

Now that the Model and View are defined, let's implement the Controller.

### Controller

The `GameOfLifeController` class will:

1. Initialize the game board using the live cell coordinates provided.
2. Update the board for a given number of steps.
3. After each update, display the board using the View.

Here's the implementation of the Controller:

```python
class GameOfLifeController:
    """Represents the controller in Conway's Game of Life.
    
    The controller acts as an intermediary between the model and the view.
    It initializes the game, updates the board, and displays the board using the view.
    """
    
    def __init__(self, initial_live_cells, steps):
        """Initialize the game with the provided live cell coordinates and steps."""
        self.model = GameOfLifeModel(initial_live_cells)
        self.steps = steps
        self.view = GameOfLifeView()
    
    def run(self):
        """Run the game for the given number of steps."""
        for step in range(self.steps + 1):  # +1 to also show the initial state
            print(f"Step: {step}")
            self.view.display(self.model.live_cells)
            if step < self.steps:  # Avoid unnecessary computation on the last step
                self.model.next_state()
```

With the Model, View, and Controller defined, let's now create a test harness to test our implementation.

### Test Harness

For testing, we'll write a function `test_game_of_life()` that will:

1. Create a list of test cases with initial live cell coordinates and the number of steps.
2. For each test case, run the game and display the board state at each step.

Let's write the test function:

```python
def test_game_of_life():
    """Test the implementation of Conway's Game of Life."""
    
    test_cases = [
        # Blinker (period 2 oscillator)
        {'initial': [(1, 0), (1, 1), (1, 2)], 'steps': 3},
        
        # Toad (period 2 oscillator)
        {'initial': [(1, 2), (2, 2), (3, 2), (0, 1), (1, 1), (2, 1)], 'steps': 3},
        
        # Glider (a pattern that moves across the board)
        {'initial': [(1, 0), (2, 1), (0, 2), (1, 2), (2, 2)], 'steps': 5},
        
        # Block (still life, doesn't change)
        {'initial': [(1, 1), (1, 2), (2, 1), (2, 2)], 'steps': 2},
        
        # Boat (still life, doesn't change)
        {'initial': [(0, 0), (1, 0), (0, 1), (2, 1), (1, 2)], 'steps': 2},
        
        # Beacon (period 2 oscillator)
        {'initial': [(0, 0), (1, 0), (0, 1), (3, 3), (2, 3), (3, 2)], 'steps': 3},
        
        # Pulsar (period 3 oscillator)
        {'initial': [(2, 4), (4, 4), (5, 4), (7, 4), (8, 4), (10, 4),
                     (4, 2), (7, 2), (10, 2), (4, 7), (7, 7), (10, 7),
                     (2, 5), (2, 7), (2, 8), (4, 10), (5, 10), (7, 10),
                     (8, 10), (10, 10), (10, 5), (10, 7), (10, 8), (12, 5),
                     (12, 7), (12, 8), (4, 5), (4, 8), (7, 5), (7, 8),
                     (10, 5), (10, 8), (12, 2), (12, 4), (12, 10)], 'steps': 4},
        
        # Lightweight spaceship (LWSS)
        {'initial': [(1, 0), (2, 0), (3, 0), (4, 0), (4, 1), (4, 2),
                     (3, 3), (1, 3), (0, 2)], 'steps': 5},
        
        # Empty board (should remain empty)
        {'initial': [], 'steps': 2},
        
        # Random pattern
        {'initial': [(2, 2), (3, 3), (4, 4), (5, 5)], 'steps': 3}
    ]
    
    for idx, test_case in enumerate(test_cases, 1):
        print(f"Test Case {idx}:\nInitial state: {test_case['initial']}\nSteps: {test_case['steps']}\n")
        controller = GameOfLifeController(test_case['initial'], test_case['steps'])
        controller.run()
        print("-" * 50)

test_game_of_life()
```

This test harness will run the game for 10 different initial configurations and display the board state at each step.

In [1]:
class GameOfLifeModel:
    """Represents the model in Conway's Game of Life.

    The model is responsible for storing the current state of the game and
    providing logic to update that state based on the rules of the game.
    """

    def __init__(self, live_cells):
        """Initialize the model with a set of live cell coordinates."""
        self.live_cells = set(live_cells)

    def is_cell_alive(self, x, y):
        """Return True if the cell at (x, y) is alive, otherwise return False."""
        return (x, y) in self.live_cells

    def live_neighbours(self, x, y):
        """Return the number of live neighbours for the cell at (x, y)."""
        offsets = [(-1, -1), (-1, 0), (-1, 1), (0, -1), (0, 1), (1, -1), (1, 0), (1, 1)]
        count = 0
        for dx, dy in offsets:
            if self.is_cell_alive(x + dx, y + dy):
                count += 1
        return count

    def next_state(self):
        """Compute the next state of the board based on the current state."""
        new_live_cells = set()

        # Define the boundary for the region that needs to be checked
        if not self.live_cells:  # If no live cells, return empty set
            return new_live_cells

        min_x = min(cell[0] for cell in self.live_cells) - 1
        max_x = max(cell[0] for cell in self.live_cells) + 1
        min_y = min(cell[1] for cell in self.live_cells) - 1
        max_y = max(cell[1] for cell in self.live_cells) + 1

        for x in range(min_x, max_x + 1):
            for y in range(min_y, max_y + 1):
                live_neigh_count = self.live_neighbours(x, y)
                if self.is_cell_alive(x, y):
                    if live_neigh_count in [2, 3]:  # Live cell with 2 or 3 live neighbours remains alive
                        new_live_cells.add((x, y))
                else:
                    if live_neigh_count == 3:  # Dead cell with 3 live neighbours becomes alive
                        new_live_cells.add((x, y))

        self.live_cells = new_live_cells


In [2]:
class GameOfLifeView:
    """Represents the view in Conway's Game of Life.

    The view is responsible for displaying the current state of the game board.
    """

    @staticmethod
    def display(live_cells):
        """Display the current state of the board using the provided live cell coordinates."""
        if not live_cells:
            print(".")
            return

        min_x = min(cell[0] for cell in live_cells)
        max_x = max(cell[0] for cell in live_cells)
        min_y = min(cell[1] for cell in live_cells)
        max_y = max(cell[1] for cell in live_cells)

        for y in range(min_y, max_y + 1):
            for x in range(min_x, max_x + 1):
                print('*' if (x, y) in live_cells else '.', end='')
            print()  # Move to the next line after each row
        print()  # Add an extra line for separation between steps


In [3]:
class GameOfLifeController:
    """Represents the controller in Conway's Game of Life.

    The controller acts as an intermediary between the model and the view.
    It initializes the game, updates the board, and displays the board using the view.
    """

    def __init__(self, initial_live_cells, steps):
        """Initialize the game with the provided live cell coordinates and steps."""
        self.model = GameOfLifeModel(initial_live_cells)
        self.steps = steps
        self.view = GameOfLifeView()

    def run(self):
        """Run the game for the given number of steps."""
        for step in range(self.steps + 1):  # +1 to also show the initial state
            print(f"Step: {step}")
            self.view.display(self.model.live_cells)
            if step < self.steps:  # Avoid unnecessary computation on the last step
                self.model.next_state()


In [4]:
def test_game_of_life():
    """Test the implementation of Conway's Game of Life."""

    test_cases = [
        # Blinker (period 2 oscillator)
        {'initial': [(1, 0), (1, 1), (1, 2)], 'steps': 3},

        # Toad (period 2 oscillator)
        {'initial': [(1, 2), (2, 2), (3, 2), (0, 1), (1, 1), (2, 1)], 'steps': 3},

        # Glider (a pattern that moves across the board)
        {'initial': [(1, 0), (2, 1), (0, 2), (1, 2), (2, 2)], 'steps': 5},

        # Block (still life, doesn't change)
        {'initial': [(1, 1), (1, 2), (2, 1), (2, 2)], 'steps': 2},

        # Boat (still life, doesn't change)
        {'initial': [(0, 0), (1, 0), (0, 1), (2, 1), (1, 2)], 'steps': 2},

        # Beacon (period 2 oscillator)
        {'initial': [(0, 0), (1, 0), (0, 1), (3, 3), (2, 3), (3, 2)], 'steps': 3},

        # Pulsar (period 3 oscillator)
        {'initial': [(2, 4), (4, 4), (5, 4), (7, 4), (8, 4), (10, 4),
                     (4, 2), (7, 2), (10, 2), (4, 7), (7, 7), (10, 7),
                     (2, 5), (2, 7), (2, 8), (4, 10), (5, 10), (7, 10),
                     (8, 10), (10, 10), (10, 5), (10, 7), (10, 8), (12, 5),
                     (12, 7), (12, 8), (4, 5), (4, 8), (7, 5), (7, 8),
                     (10, 5), (10, 8), (12, 2), (12, 4), (12, 10)], 'steps': 4},

        # Lightweight spaceship (LWSS)
        {'initial': [(1, 0), (2, 0), (3, 0), (4, 0), (4, 1), (4, 2),
                     (3, 3), (1, 3), (0, 2)], 'steps': 5},

        # Empty board (should remain empty)
        {'initial': [], 'steps': 2},

        # Random pattern
        {'initial': [(2, 2), (3, 3), (4, 4), (5, 5)], 'steps': 3}
    ]

    for idx, test_case in enumerate(test_cases, 1):
        print(f"Test Case {idx}:\nInitial state: {test_case['initial']}\nSteps: {test_case['steps']}\n")
        controller = GameOfLifeController(test_case['initial'], test_case['steps'])
        controller.run()
        print("-" * 50)

test_game_of_life()


Test Case 1:
Initial state: [(1, 0), (1, 1), (1, 2)]
Steps: 3

Step: 0
*
*
*

Step: 1
***

Step: 2
*
*
*

Step: 3
***

--------------------------------------------------
Test Case 2:
Initial state: [(1, 2), (2, 2), (3, 2), (0, 1), (1, 1), (2, 1)]
Steps: 3

Step: 0
***.
.***

Step: 1
.*..
*..*
*..*
..*.

Step: 2
***.
.***

Step: 3
.*..
*..*
*..*
..*.

--------------------------------------------------
Test Case 3:
Initial state: [(1, 0), (2, 1), (0, 2), (1, 2), (2, 2)]
Steps: 5

Step: 0
.*.
..*
***

Step: 1
*.*
.**
.*.

Step: 2
..*
*.*
.**

Step: 3
*..
.**
**.

Step: 4
.*.
..*
***

Step: 5
*.*
.**
.*.

--------------------------------------------------
Test Case 4:
Initial state: [(1, 1), (1, 2), (2, 1), (2, 2)]
Steps: 2

Step: 0
**
**

Step: 1
**
**

Step: 2
**
**

--------------------------------------------------
Test Case 5:
Initial state: [(0, 0), (1, 0), (0, 1), (2, 1), (1, 2)]
Steps: 2

Step: 0
**.
*.*
.*.

Step: 1
**.
*.*
.*.

Step: 2
**.
*.*
.*.

-------------------------------

In [5]:
import time

class AnimatedGameOfLifeController:
    """Represents the animated controller in Conway's Game of Life.

    This controller acts as an intermediary between the model and the view.
    It initializes the game, updates the board, and displays the board using the view,
    with a delay between each step to simulate an animation.
    """

    def __init__(self, initial_live_cells, steps, delay=0.5):
        """Initialize the game with the provided live cell coordinates and steps."""
        self.model = GameOfLifeModel(initial_live_cells)
        self.steps = steps
        self.view = GameOfLifeView()
        self.delay = delay  # Time delay between each step in seconds

    def run(self):
        """Run the game for the given number of steps with a delay between each step."""
        for step in range(self.steps + 1):  # +1 to also show the initial state
            print(f"Step: {step}")
            self.view.display(self.model.live_cells)
            if step < self.steps:  # Avoid unnecessary computation on the last step
                self.model.next_state()
                time.sleep(self.delay)  # Introduce delay between steps


In [6]:
# Redefining the Model and View

class GameOfLifeModel:
    """Represents the model in Conway's Game of Life."""
    def __init__(self, live_cells):
        self.live_cells = set(live_cells)

    def is_cell_alive(self, x, y):
        return (x, y) in self.live_cells

    def live_neighbours(self, x, y):
        offsets = [(-1, -1), (-1, 0), (-1, 1), (0, -1), (0, 1), (1, -1), (1, 0), (1, 1)]
        count = 0
        for dx, dy in offsets:
            if self.is_cell_alive(x + dx, y + dy):
                count += 1
        return count

    def next_state(self):
        new_live_cells = set()
        if not self.live_cells:
            return new_live_cells
        min_x = min(cell[0] for cell in self.live_cells) - 1
        max_x = max(cell[0] for cell in self.live_cells) + 1
        min_y = min(cell[1] for cell in self.live_cells) - 1
        max_y = max(cell[1] for cell in self.live_cells) + 1
        for x in range(min_x, max_x + 1):
            for y in range(min_y, max_y + 1):
                live_neigh_count = self.live_neighbours(x, y)
                if self.is_cell_alive(x, y):
                    if live_neigh_count in [2, 3]:
                        new_live_cells.add((x, y))
                else:
                    if live_neigh_count == 3:
                        new_live_cells.add((x, y))
        self.live_cells = new_live_cells

class GameOfLifeView:
    """Represents the view in Conway's Game of Life."""
    @staticmethod
    def display(live_cells):
        if not live_cells:
            print(".")
            return
        min_x = min(cell[0] for cell in live_cells)
        max_x = max(cell[0] for cell in live_cells)
        min_y = min(cell[1] for cell in live_cells)
        max_y = max(cell[1] for cell in live_cells)
        for y in range(min_y, max_y + 1):
            for x in range(min_x, max_x + 1):
                print('*' if (x, y) in live_cells else '.', end='')
            print()
        print()

# Running the animation again
glider = [(1, 0), (2, 1), (0, 2), (1, 2), (2, 2)]
controller = AnimatedGameOfLifeController(glider, 20, 0.5)
controller.run()


Step: 0
.*.
..*
***

Step: 1
*.*
.**
.*.

Step: 2
..*
*.*
.**

Step: 3
*..
.**
**.

Step: 4
.*.
..*
***

Step: 5
*.*
.**
.*.

Step: 6
..*
*.*
.**

Step: 7
*..
.**
**.

Step: 8
.*.
..*
***

Step: 9
*.*
.**
.*.

Step: 10
..*
*.*
.**

Step: 11
*..
.**
**.

Step: 12
.*.
..*
***

Step: 13
*.*
.**
.*.

Step: 14
..*
*.*
.**

Step: 15
*..
.**
**.

Step: 16
.*.
..*
***

Step: 17
*.*
.**
.*.

Step: 18
..*
*.*
.**

Step: 19
*..
.**
**.

Step: 20
.*.
..*
***

